## Import

In [30]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

from tensorflow import keras
from tensorflow.keras import layers

In [31]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [32]:
DIR_PATH = "gdrive/MyDrive/ChamoAnalytics"

In [137]:
df = pd.read_csv(f"{DIR_PATH}/master_dataset.csv", na_values=['-1'])

## Preprocessing

In [34]:
# df[["date"]] = df[["date"]].apply(pd.to_datetime)

In [138]:
df = df.replace('.', np.nan)
df[[" us_bank_rate"]] = df[[" us_bank_rate"]].apply(pd.to_numeric)
df = df.rename(columns={' us_bank_rate': 'us_bank_rate'})

In [139]:
df.dtypes

date                            object
rate                           float64
sent_fin_us_uncertainty        float64
sent_fin_us_litigious          float64
sent_fin_us_strong_modal       float64
sent_fin_us_weak_modal         float64
sent_fin_us_constraining       float64
sent_fin_us_optimistic         float64
sent_fin_can_uncertainty       float64
sent_fin_can_litigious         float64
sent_fin_can_strong_modal      float64
sent_fin_can_weak_modal        float64
sent_fin_can_constraining      float64
sent_fin_can_optimistic        float64
ti_rate_rsi_14                 float64
ti_rate_stochrsi_14            float64
ti_rate_macd_12_26             float64
ti_rate_adx_14                 float64
ti_rate_williams_%R            float64
ti_rate_cci                    float64
ti_rate_atr                    float64
ti_rate_utlimate_oscillator    float64
ti_rate_roc                    float64
interest_rate_can              float64
index_W.BCPI                   float64
index_W.BCNE             

In [140]:
df.size

1028250

In [141]:
df['y_exp'] = np.full(df.shape[0], -1)

for i, row in df.iterrows():
    try:
        two_weeks_from_current_date = datetime.datetime.strptime(row.date, "%Y-%m-%d") + datetime.timedelta(weeks=2)

        futur_rate = list(df.loc[df['date'] == str(two_weeks_from_current_date.date())].rate)[0]
        df.loc[i, 'y_exp' ] = futur_rate
    except: 
        break

df = df.loc[df.y_exp >= 0]

In [142]:
df = df.dropna()
df = df.reset_index(drop=True)

In [143]:
df.size

307924

In [144]:
def to_sequences(seq_size, obs, y):
    x = []
    y_out = []

    for i in range(len(obs)-seq_size):
        #print(i)
        window = obs[i:(i+seq_size)]
        after_window = y[i+seq_size - 1]
        window = [x for x in window]
        x.append(window)
        y_out.append(after_window)
        
    return np.array(x), np.array(y_out)

In [145]:
def prepare_data(df, cols_to_keep, seq_size):
    cutting_date = '2022-08-01'
    df_train = df[df['date'] < cutting_date]
    df_test = df[df['date'] >= cutting_date]

    x_train = df_train[cols_to_keep].to_numpy()
    y_train = df_train['y_exp'].to_numpy()

    x_test = df_test[cols_to_keep].to_numpy()
    y_test = df_test['y_exp'].to_numpy()

    x_train, y_train = to_sequences(seq_size, x_train, y_train)
    x_test, y_test = to_sequences(seq_size, x_test, y_test)

    print("Train: {}".format(x_train.shape))
    print("Test: {}".format(x_test.shape))

    return x_train, y_train, x_test, y_test

## Modeling

In [146]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)

    return x + res

In [147]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    
    return keras.Model(inputs, outputs)

## Training

In [148]:
def train(x_train, y_train):
    input_shape = x_train.shape[1:]

    model = build_model(
        input_shape,
        head_size=256,
        num_heads=4,
        ff_dim=4,
        num_transformer_blocks=4,
        mlp_units=[128],
        mlp_dropout=0.4,
        dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4)
    )

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    model.fit(
        x_train,
        y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=64,
        callbacks=callbacks,
    )

    return model

## Evaluating

In [156]:
from copy import deepcopy

def run(df, cols_to_keep, seq_size, nb_trials=5):
    x_train, y_train, x_test, y_test = prepare_data(df, cols_to_keep, seq_size)

    errors = []
    best_model = None
    smallest_error = None

    for _ in range(nb_trials):
        model = train(x_train, y_train)

        error = model.evaluate(x_test, y_test, verbose=1)

        if not smallest_error or smallest_error > error:
            best_model = deepcopy(model)
            smallest_error = error
        errors.append(error)

    
    print('mean: ', np.mean(errors), 'var: ', np.var(errors))
    print('smallest error: ', smallest_error)

    return best_model

### rate

In [150]:
cols_to_keep = ['rate']
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (6615, 20, 1)
Test: (39, 20, 1)
Epoch 1/200
83/83 [==============================] - 6s 24ms/step - loss: 1.2273 - val_loss: 0.0585
Epoch 2/200
83/83 [==============================] - 1s 16ms/step - loss: 0.2268 - val_loss: 0.0103
Epoch 3/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1573 - val_loss: 0.0050
Epoch 4/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1304 - val_loss: 0.0013
Epoch 5/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1249 - val_loss: 0.0021
Epoch 6/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1115 - val_loss: 0.0014
Epoch 7/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1059 - val_loss: 0.0014
Epoch 8/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1003 - val_loss: 0.0013
Epoch 9/200
83/83 [==============================] - 1s 16ms/step - loss: 0.0911 - val_loss: 0.0022
Epoch 10/200
83/83 [==============================] - 1s 17ms

[0.001657019485719502, 0.0025986202526837587]

### sent_bert_us_fin

In [152]:
cols_to_keep = ['sent_bert_us_fin']
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (6615, 20, 1)
Test: (39, 20, 1)
Epoch 1/200
83/83 [==============================] - 6s 23ms/step - loss: 1.0684 - val_loss: 0.0104
Epoch 2/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1808 - val_loss: 0.0217
Epoch 3/200
83/83 [==============================] - 2s 18ms/step - loss: 0.1425 - val_loss: 0.0237
Epoch 4/200
83/83 [==============================] - 2s 24ms/step - loss: 0.1256 - val_loss: 0.0242
Epoch 5/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1216 - val_loss: 0.0240
Epoch 6/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1148 - val_loss: 0.0248
Epoch 7/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1083 - val_loss: 0.0249
Epoch 8/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1067 - val_loss: 0.0246
Epoch 9/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1038 - val_loss: 0.0270
Epoch 10/200
83/83 [==============================] - 1s 17ms

[0.010098583064973354, 0.03212585672736168]

### sent_bert_us_gen

In [153]:
cols_to_keep = ['sent_bert_us_gen']
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (6615, 20, 1)
Test: (39, 20, 1)
Epoch 1/200
83/83 [==============================] - 6s 23ms/step - loss: 0.9155 - val_loss: 0.0044
Epoch 2/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1932 - val_loss: 0.0113
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1515 - val_loss: 0.0081
Epoch 4/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1318 - val_loss: 0.0082
Epoch 5/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1210 - val_loss: 0.0069
Epoch 6/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1128 - val_loss: 0.0080
Epoch 7/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1107 - val_loss: 0.0093
Epoch 8/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1093 - val_loss: 0.0074
Epoch 9/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1041 - val_loss: 0.0078
Epoch 10/200
83/83 [==============================] - 1s 16ms

[0.005119812209159136, 0.004135191906243563]

### sent_bert_can_fin_sentiment

In [154]:
cols_to_keep = ['sent_bert_can_fin_sentiment']
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (6615, 20, 1)
Test: (39, 20, 1)
Epoch 1/200
83/83 [==============================] - 6s 23ms/step - loss: 0.2566 - val_loss: 0.0390
Epoch 2/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1727 - val_loss: 0.0494
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1600 - val_loss: 0.0539
Epoch 4/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1525 - val_loss: 0.0476
Epoch 5/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1405 - val_loss: 0.0378
Epoch 6/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1332 - val_loss: 0.0393
Epoch 7/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1245 - val_loss: 0.0515
Epoch 8/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1166 - val_loss: 0.0357
Epoch 9/200
83/83 [==============================] - 1s 17ms/step - loss: 0.1095 - val_loss: 0.0342
Epoch 10/200
83/83 [==============================] - 1s 16ms

[0.006889623124152422, 0.015397908166050911]

### sent_bert_can_gen_sentiment

In [155]:
cols_to_keep = ['sent_bert_can_gen_sentiment']
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (6615, 20, 1)
Test: (39, 20, 1)
Epoch 1/200
83/83 [==============================] - 5s 23ms/step - loss: 0.8295 - val_loss: 0.0511
Epoch 2/200
83/83 [==============================] - 1s 17ms/step - loss: 0.2267 - val_loss: 0.0395
Epoch 3/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1793 - val_loss: 0.0394
Epoch 4/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1595 - val_loss: 0.0364
Epoch 5/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1451 - val_loss: 0.0551
Epoch 6/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1348 - val_loss: 0.0500
Epoch 7/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1311 - val_loss: 0.0373
Epoch 8/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1255 - val_loss: 0.0497
Epoch 9/200
83/83 [==============================] - 1s 16ms/step - loss: 0.1230 - val_loss: 0.0393
Epoch 10/200
83/83 [==============================] - 1s 16ms

[0.04243490472435951, 0.04297584667801857]

### rate, sent_bert_us_fin

In [151]:
cols_to_keep = ['rate', 'sent_bert_us_fin']
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (6615, 20, 2)
Test: (39, 20, 2)
Epoch 1/200
83/83 [==============================] - 6s 25ms/step - loss: 3.6858 - val_loss: 1.8957
Epoch 2/200
83/83 [==============================] - 2s 20ms/step - loss: 1.8939 - val_loss: 1.5181
Epoch 3/200
83/83 [==============================] - 1s 18ms/step - loss: 0.9651 - val_loss: 0.1151
Epoch 4/200
83/83 [==============================] - 2s 18ms/step - loss: 0.3991 - val_loss: 0.0780
Epoch 5/200
83/83 [==============================] - 1s 18ms/step - loss: 0.3766 - val_loss: 0.0089
Epoch 6/200
83/83 [==============================] - 2s 18ms/step - loss: 0.3571 - val_loss: 0.0058
Epoch 7/200
83/83 [==============================] - 1s 17ms/step - loss: 0.2910 - val_loss: 0.0089
Epoch 8/200
83/83 [==============================] - 2s 19ms/step - loss: 0.2620 - val_loss: 0.0102
Epoch 9/200
83/83 [==============================] - 2s 27ms/step - loss: 0.2500 - val_loss: 0.0147
Epoch 10/200
83/83 [==============================] - 1s 18ms

[0.015270878560841084, 0.06985127925872803]

### rate, sent_bert_us_fin, sent_bert_us_gen

In [135]:
cols_to_keep = ['rate', 'sent_bert_us_fin', 'sent_bert_us_gen']
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (7305, 20, 3)
Test: (39, 20, 3)
Epoch 1/200
92/92 [==============================] - 7s 23ms/step - loss: 1.9266 - val_loss: 0.2375
Epoch 2/200
92/92 [==============================] - 2s 18ms/step - loss: 0.4979 - val_loss: 0.0154
Epoch 3/200
92/92 [==============================] - 2s 18ms/step - loss: 0.3499 - val_loss: 0.0303
Epoch 4/200
92/92 [==============================] - 2s 17ms/step - loss: 0.2764 - val_loss: 0.0280
Epoch 5/200
92/92 [==============================] - 2s 18ms/step - loss: 0.2459 - val_loss: 0.0218
Epoch 6/200
92/92 [==============================] - 2s 17ms/step - loss: 0.2158 - val_loss: 0.0318
Epoch 7/200
92/92 [==============================] - 2s 17ms/step - loss: 0.1844 - val_loss: 0.0247
Epoch 8/200
92/92 [==============================] - 2s 24ms/step - loss: 0.1661 - val_loss: 0.0338
Epoch 9/200
92/92 [==============================] - 2s 18ms/step - loss: 0.1475 - val_loss: 0.0450
Epoch 10/200
92/92 [==============================] - 2s 17ms

[0.006841078866273165, 0.0014830788131803274]

### all sentiments only

In [173]:
cols_to_keep = [
    'sent_fin_can_litigious',
    'sent_fin_us_strong_modal',
    'sent_fin_us_uncertainty',
    'sent_fin_can_uncertainty',
    'sent_fin_us_weak_modal',
    'sent_bert_can_fin_sentiment',
    'sent_bert_us_gen',
    'sent_bert_can_gen_sentiment',
    'sent_fin_us_litigious',
    'sent_bert_us_fin',
    'sent_fin_can_strong_modal',
    'sent_fin_can_weak_modal',
    'sent_fin_can_optimistic',
    'sent_fin_us_optimistic',
    'sent_fin_can_constraining',
    'sent_fin_us_constraining',
]

seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

Train: (6615, 20, 16)
Test: (39, 20, 16)
Epoch 1/200
83/83 [==============================] - 6s 24ms/step - loss: 3.5479 - val_loss: 1.1932
Epoch 2/200
83/83 [==============================] - 1s 18ms/step - loss: 1.4194 - val_loss: 0.8919
Epoch 3/200
83/83 [==============================] - 2s 18ms/step - loss: 1.1504 - val_loss: 0.6671
Epoch 4/200
83/83 [==============================] - 1s 18ms/step - loss: 0.9427 - val_loss: 0.5551
Epoch 5/200
83/83 [==============================] - 1s 18ms/step - loss: 0.7926 - val_loss: 0.4186
Epoch 6/200
83/83 [==============================] - 1s 17ms/step - loss: 0.7205 - val_loss: 0.3660
Epoch 7/200
83/83 [==============================] - 1s 18ms/step - loss: 0.5966 - val_loss: 0.3247
Epoch 8/200
83/83 [==============================] - 1s 17ms/step - loss: 0.5800 - val_loss: 0.2197
Epoch 9/200
83/83 [==============================] - 1s 18ms/step - loss: 0.5260 - val_loss: 0.2341
Epoch 10/200
83/83 [==============================] - 2s 18

Epoch 1/200
83/83 [==============================] - 5s 24ms/step - loss: 3.9895 - val_loss: 1.0087
Epoch 2/200
83/83 [==============================] - 1s 18ms/step - loss: 1.7593 - val_loss: 0.7485
Epoch 3/200
83/83 [==============================] - 2s 18ms/step - loss: 1.4070 - val_loss: 0.6258
Epoch 4/200
83/83 [==============================] - 1s 18ms/step - loss: 1.2048 - val_loss: 0.5415
Epoch 5/200
83/83 [==============================] - 2s 18ms/step - loss: 1.0430 - val_loss: 0.4648
Epoch 6/200
83/83 [==============================] - 1s 18ms/step - loss: 0.9287 - val_loss: 0.4042
Epoch 7/200
83/83 [==============================] - 2s 18ms/step - loss: 0.8258 - val_loss: 0.3392
Epoch 8/200
83/83 [==============================] - 1s 18ms/step - loss: 0.6946 - val_loss: 0.2783
Epoch 9/200
83/83 [==============================] - 1s 18ms/step - loss: 0.6027 - val_loss: 0.2363
Epoch 10/200
83/83 [==============================] - 2s 18ms/step - loss: 0.5408 - val_loss: 0.2138

Epoch 1/200
83/83 [==============================] - 5s 24ms/step - loss: 4.5580 - val_loss: 1.1892
Epoch 2/200
83/83 [==============================] - 2s 19ms/step - loss: 1.9622 - val_loss: 0.9004
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 1.5201 - val_loss: 0.7711
Epoch 4/200
83/83 [==============================] - 1s 18ms/step - loss: 1.2905 - val_loss: 0.6589
Epoch 5/200
83/83 [==============================] - 1s 17ms/step - loss: 1.1427 - val_loss: 0.5793
Epoch 6/200
83/83 [==============================] - 1s 17ms/step - loss: 0.9800 - val_loss: 0.4525
Epoch 7/200
83/83 [==============================] - 1s 18ms/step - loss: 0.8783 - val_loss: 0.3933
Epoch 8/200
83/83 [==============================] - 1s 18ms/step - loss: 0.7451 - val_loss: 0.3405
Epoch 9/200
83/83 [==============================] - 1s 18ms/step - loss: 0.6797 - val_loss: 0.3110
Epoch 10/200
83/83 [==============================] - 1s 17ms/step - loss: 0.5814 - val_loss: 0.2571

### interest_rate_can

In [169]:
cols_to_keep = [
    'interest_rate_can'
]

seq_size = 20
best_model = run(df, cols_to_keep, seq_size, nb_trials=1)

Train: (6615, 20, 1)
Test: (39, 20, 1)
Epoch 1/200
83/83 [==============================] - 7s 39ms/step - loss: 1.8644 - val_loss: 0.6636
Epoch 2/200
83/83 [==============================] - 1s 17ms/step - loss: 0.9623 - val_loss: 0.5619
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 0.8336 - val_loss: 0.5160
Epoch 4/200
83/83 [==============================] - 1s 17ms/step - loss: 0.7219 - val_loss: 0.4684
Epoch 5/200
83/83 [==============================] - 1s 16ms/step - loss: 0.6277 - val_loss: 0.4493
Epoch 6/200
83/83 [==============================] - 1s 16ms/step - loss: 0.5118 - val_loss: 0.4484
Epoch 7/200
83/83 [==============================] - 1s 17ms/step - loss: 0.4559 - val_loss: 0.4602
Epoch 8/200
83/83 [==============================] - 1s 17ms/step - loss: 0.4014 - val_loss: 0.4714
Epoch 9/200
83/83 [==============================] - 1s 16ms/step - loss: 0.3387 - val_loss: 0.4520
Epoch 10/200
83/83 [==============================] - 1s 16ms

mean:  0.005863863509148359 var:  0.0
smallest error:  0.005863863509148359


### rate, interest_rate_can

In [171]:
cols_to_keep = [
    'interest_rate_can',
    'rate'
]

seq_size = 20
best_model = run(df, cols_to_keep, seq_size, nb_trials=1)

Train: (6615, 20, 2)
Test: (39, 20, 2)
Epoch 1/200
83/83 [==============================] - 6s 25ms/step - loss: 1.2412 - val_loss: 1.1066
Epoch 2/200
83/83 [==============================] - 2s 18ms/step - loss: 0.7756 - val_loss: 0.9025
Epoch 3/200
83/83 [==============================] - 2s 19ms/step - loss: 0.6103 - val_loss: 0.6286
Epoch 4/200
83/83 [==============================] - 2s 19ms/step - loss: 0.4642 - val_loss: 0.6164
Epoch 5/200
83/83 [==============================] - 1s 18ms/step - loss: 0.3877 - val_loss: 0.5670
Epoch 6/200
83/83 [==============================] - 2s 18ms/step - loss: 0.3065 - val_loss: 0.5565
Epoch 7/200
83/83 [==============================] - 1s 18ms/step - loss: 0.2619 - val_loss: 0.5111
Epoch 8/200
83/83 [==============================] - 2s 18ms/step - loss: 0.2128 - val_loss: 0.4965
Epoch 9/200
83/83 [==============================] - 2s 18ms/step - loss: 0.1860 - val_loss: 0.4814
Epoch 10/200
83/83 [==============================] - 2s 18ms

mean:  0.07451485842466354 var:  0.0
smallest error:  0.07451485842466354


### Top-3 of the features based on LSTM results

In [170]:
cols_to_keep = [
    'interest_rate_can',
    'index_WGTS.BRENT',
    'rate'
]

seq_size = 20
best_model = run(df, cols_to_keep, seq_size, nb_trials=1)

Train: (6615, 20, 3)
Test: (39, 20, 3)
Epoch 1/200
83/83 [==============================] - 6s 25ms/step - loss: 1.4057 - val_loss: 0.9064
Epoch 2/200
83/83 [==============================] - 2s 20ms/step - loss: 0.6250 - val_loss: 1.1306
Epoch 3/200
83/83 [==============================] - 2s 19ms/step - loss: 0.4759 - val_loss: 0.8635
Epoch 4/200
83/83 [==============================] - 2s 19ms/step - loss: 0.3797 - val_loss: 0.8404
Epoch 5/200
83/83 [==============================] - 2s 22ms/step - loss: 0.3136 - val_loss: 0.7682
Epoch 6/200
83/83 [==============================] - 2s 18ms/step - loss: 0.2757 - val_loss: 0.5667
Epoch 7/200
83/83 [==============================] - 2s 19ms/step - loss: 0.2410 - val_loss: 0.4759
Epoch 8/200
83/83 [==============================] - 2s 19ms/step - loss: 0.2232 - val_loss: 0.5079
Epoch 9/200
83/83 [==============================] - 2s 19ms/step - loss: 0.2139 - val_loss: 0.4816
Epoch 10/200
83/83 [==============================] - 2s 18ms

mean:  0.26858970522880554 var:  0.0
smallest error:  0.26858970522880554


### Top-5 of the features based on LSTM results

In [174]:
cols_to_keep = [
    'interest_rate_can',
    'index_WGTS.BRENT',
    'rate',
    'ti_rate_rsi_14',
    'sent_fin_can_litigious',
]

seq_size = 20
best_model = run(df, cols_to_keep, seq_size, nb_trials=5)

Train: (6615, 20, 5)
Test: (39, 20, 5)
Epoch 1/200
83/83 [==============================] - 5s 23ms/step - loss: 20.2934 - val_loss: 1.6423
Epoch 2/200
83/83 [==============================] - 2s 19ms/step - loss: 6.5104 - val_loss: 1.3185
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 3.0064 - val_loss: 0.7361
Epoch 4/200
83/83 [==============================] - 1s 17ms/step - loss: 2.1551 - val_loss: 0.5306
Epoch 5/200
83/83 [==============================] - 1s 17ms/step - loss: 1.8935 - val_loss: 0.4247
Epoch 6/200
83/83 [==============================] - 1s 17ms/step - loss: 1.6762 - val_loss: 0.3573
Epoch 7/200
83/83 [==============================] - 1s 17ms/step - loss: 1.5756 - val_loss: 0.3215
Epoch 8/200
83/83 [==============================] - 1s 17ms/step - loss: 1.3729 - val_loss: 0.2769
Epoch 9/200
83/83 [==============================] - 1s 17ms/step - loss: 1.2193 - val_loss: 0.2482
Epoch 10/200
83/83 [==============================] - 1s 17m

Epoch 1/200
83/83 [==============================] - 5s 24ms/step - loss: 43.1894 - val_loss: 0.4857
Epoch 2/200
83/83 [==============================] - 1s 17ms/step - loss: 12.5584 - val_loss: 0.6667
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 6.2968 - val_loss: 1.0401
Epoch 4/200
83/83 [==============================] - 1s 17ms/step - loss: 3.5837 - val_loss: 1.1081
Epoch 5/200
83/83 [==============================] - 1s 18ms/step - loss: 2.7234 - val_loss: 0.5824
Epoch 6/200
83/83 [==============================] - 1s 17ms/step - loss: 2.2885 - val_loss: 0.3933
Epoch 7/200
83/83 [==============================] - 1s 18ms/step - loss: 2.0555 - val_loss: 0.3309
Epoch 8/200
83/83 [==============================] - 1s 18ms/step - loss: 1.8533 - val_loss: 0.2853
Epoch 9/200
83/83 [==============================] - 1s 18ms/step - loss: 1.7812 - val_loss: 0.2601
Epoch 10/200
83/83 [==============================] - 1s 17ms/step - loss: 1.6915 - val_loss: 0.25

Epoch 1/200
83/83 [==============================] - 5s 23ms/step - loss: 37.0933 - val_loss: 0.5698
Epoch 2/200
83/83 [==============================] - 1s 18ms/step - loss: 9.2941 - val_loss: 0.4343
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 4.7728 - val_loss: 0.4460
Epoch 4/200
83/83 [==============================] - 2s 18ms/step - loss: 2.5378 - val_loss: 0.3954
Epoch 5/200
83/83 [==============================] - 1s 18ms/step - loss: 1.8402 - val_loss: 0.3377
Epoch 6/200
83/83 [==============================] - 1s 17ms/step - loss: 1.7304 - val_loss: 0.2871
Epoch 7/200
83/83 [==============================] - 1s 17ms/step - loss: 1.6530 - val_loss: 0.2978
Epoch 8/200
83/83 [==============================] - 1s 18ms/step - loss: 1.5468 - val_loss: 0.2778
Epoch 9/200
83/83 [==============================] - 1s 17ms/step - loss: 1.4019 - val_loss: 0.2647
Epoch 10/200
83/83 [==============================] - 1s 17ms/step - loss: 1.3105 - val_loss: 0.276

In [175]:
best_model.save(f"{DIR_PATH}/best_transformer_model")

### Quarter of the features based on LSTM results

In [172]:
cols_to_keep = [
    'interest_rate_can',
    'index_WGTS.BRENT',
    'rate',
    'ti_rate_rsi_14',
    'sent_fin_can_litigious',
    'ti_rate_stochrsi_14',
    'sent_fin_us_strong_modal',
    'ti_rate_williams_%R',
    'index_WGTS.COAL',
    'ti_rate_macd_12_26',
    'sent_fin_us_uncertainty',
]

seq_size = 20
best_model = run(df, cols_to_keep, seq_size, nb_trials=1)

Train: (6615, 20, 11)
Test: (39, 20, 11)
Epoch 1/200
83/83 [==============================] - 6s 26ms/step - loss: 4.6029 - val_loss: 0.8803
Epoch 2/200
83/83 [==============================] - 2s 20ms/step - loss: 2.7006 - val_loss: 0.6394
Epoch 3/200
83/83 [==============================] - 1s 17ms/step - loss: 2.1274 - val_loss: 0.4879
Epoch 4/200
83/83 [==============================] - 1s 18ms/step - loss: 1.6125 - val_loss: 0.3489
Epoch 5/200
83/83 [==============================] - 1s 17ms/step - loss: 1.1843 - val_loss: 0.2162
Epoch 6/200
83/83 [==============================] - 1s 17ms/step - loss: 0.8695 - val_loss: 0.1580
Epoch 7/200
83/83 [==============================] - 1s 18ms/step - loss: 0.7155 - val_loss: 0.1304
Epoch 8/200
83/83 [==============================] - 1s 17ms/step - loss: 0.6208 - val_loss: 0.1085
Epoch 9/200
83/83 [==============================] - 1s 18ms/step - loss: 0.5338 - val_loss: 0.0966
Epoch 10/200
83/83 [==============================] - 2s 21

mean:  0.004362407606095076 var:  0.0
smallest error:  0.004362407606095076


### Half of the features based on LSTM results

In [166]:
cols_to_keep = [
    'interest_rate_can',
    'index_WGTS.BRENT',
    'rate',
    'ti_rate_rsi_14',
    'sent_fin_can_litigious',
    'ti_rate_stochrsi_14',
    'sent_fin_us_strong_modal',
    'ti_rate_williams_%R',
    'index_WGTS.COAL',
    'ti_rate_macd_12_26',
    'sent_fin_us_uncertainty',
    'sent_fin_can_uncertainty',
    'index_W.FISH',
    'ti_rate_atr',
    'sent_fin_us_weak_modal',
    'sent_bert_can_fin_sentiment',
    'index_WGTS.WCC',
    'index_W.ENER',
    'sent_bert_us_gen',
    'sent_bert_can_gen_sentiment',
    'sent_fin_us_litigious',
    'sent_bert_us_fin'
]
seq_size = 20
best_model = run(df, cols_to_keep, seq_size, nb_trials=1)

Train: (6615, 20, 22)
Test: (39, 20, 22)
Epoch 1/200
83/83 [==============================] - 6s 25ms/step - loss: 1972.0529 - val_loss: 3.7877
Epoch 2/200
83/83 [==============================] - 2s 19ms/step - loss: 704.2396 - val_loss: 2.5931
Epoch 3/200
83/83 [==============================] - 2s 18ms/step - loss: 483.5887 - val_loss: 3.0118
Epoch 4/200
83/83 [==============================] - 2s 19ms/step - loss: 265.6248 - val_loss: 2.6325
Epoch 5/200
83/83 [==============================] - 2s 19ms/step - loss: 121.8922 - val_loss: 2.1477
Epoch 6/200
83/83 [==============================] - 2s 19ms/step - loss: 75.0638 - val_loss: 2.2437
Epoch 7/200
83/83 [==============================] - 2s 22ms/step - loss: 53.4006 - val_loss: 1.7698
Epoch 8/200
83/83 [==============================] - 2s 21ms/step - loss: 46.1519 - val_loss: 1.4767
Epoch 9/200
83/83 [==============================] - 2s 30ms/step - loss: 33.2890 - val_loss: 1.5036
Epoch 10/200
83/83 [========================

mean:  0.27022773027420044 var:  0.0
smallest error:  0.27022773027420044


### All

In [22]:
cols_to_keep = df.drop(columns=['date', 'y_exp']).columns
seq_size = 20
best_model = run(df, cols_to_keep, seq_size)

2/2 [==============================] - 0s 34ms/step - loss: 0.2937


0.29366597533226013

### Best Tranformer Model

Top-5 of the best features from LSTM evaluation

error: 0.0005021084798499942